In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from datetime import datetime 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [5]:
%run ../pylib/metaphone.py


**********************************************************************
File "/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/tablecloth.io/tablecloth.io-storytent-201811/pylib/metaphone.py", line 19, in __main__
Failed example:
    dm(u'Bartoš'), dm(u'Bartosz'), dm(u'Bartosch'), dm(u'Bartos')
Expected:
    (('PRT', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
Got:
    (('PRTS', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
**********************************************************************
1 items had failures:
   1 of   4 in __main__
***Test Failed*** 1 failures.


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

In [3]:
ut = pkg.resource('users').dataframe()

ut['sound_key'] = ut.apply(lambda r: "{}/{}".format(r.last_dm,r.first_dm ), axis=1)

ut.head()

,clientid,form,last,first,last_dm,first_dm,sex,grade,birthday,sound_key
0,3265196f0219735433173d32066314ae,Checkin,Polendo,Greyson,PLNT,KRSN,Boy,0,2015-08-27,PLNT/KRSN
1,669bebe1bcc2051720ee3228f024b90a,Checkin,Li,Charlotte,L,XRLT,Girl,0,2017-01-11,L/XRLT
2,47835a5a28da0fe0fe215d61b7ee7212,Checkin,Li,Jj,L,J,Boy,4,2018-04-11,L/J
3,a98f4d648e07753880157b46ac339f07,Checkin,Mora,Zolin,MR,SLN,Boy,0,2014-04-25,MR/SLN
4,d5c0a163d18458da599345975cbe155c,Checkin,Mora,Tito,MR,TT,Boy,2,2011-03-22,MR/TT


,type,location,time_in,time_out,duration,feeling_in,feeling_out,first_name,last_name,full_name,name_key,user_id,session_id,checkin_row_no,checkout_row_no,sound_key
0,c,ElCajon,2017-10-25 15:29:23,2017-10-25 16:53:13,84,heart eyes,heart eyes,Aelen,Putrus,,Aelen-Putrus-2011-04-13,5b4b7d42ba8715bf88c384694ecda9c7,a5990f9e-11a9-4e9a-9d8c-ca96cc5a0a87,1,6658,PTRS/ALN
1,i,CityHeights,2018-09-01 08:19:19,,,heart eyes,,Benjamin,Kolee,,Benjamin-Kolee-2018-08-21,7e8c8cdcdcf72e219ede9108da2aef8b,13f0c859-4ab3-44c2-a234-170d16c5f088,2,,KL/PNJMN
2,i,CityHeights,2018-09-01 08:20:33,,,smiling face,,Mary,Kolee,,Mary-Kolee-na,1666804a08d199c707754606e3c988b5,9c800f35-6c93-4587-a179-ef8eb5aaa421,3,,KL/MR
3,i,Vista,2018-09-01 08:21:44,,,smiling face,,Elias,Cheng,,Elias-Cheng-2010-07-12,157fa4828f5e662d869a5ab66d0ac3c7,211f8a88-4bba-49af-a188-b29b38a5124b,4,,XNK/ALS
4,i,CityHeights,2018-09-01 08:22:30,,,smiling face,,Javier,Mendez,,Javier-Mendez-2014-02-16,256399200dd111398947757ce65bbdad,26fae1c1-6df3-47d6-96a7-7d28201d49fa,5,,MNTS/JF


In [4]:
t = ut[ut.birthday.notnull()][['sound_key', 'first','last', 'first_dm','last_dm','birthday']]\
    .set_index(['first_dm','last_dm']).drop_duplicates().copy()

t['bd'] = t.birthday.apply(lambda x: str(x))

g = t.groupby(['first_dm','last_dm'])
t['allocated']  = False

# Immediately allocate everything
for idx, group in g:
   
    if group.count()['bd'] == 1:
        try:
            t.loc[idx,'allocated'] = True
        except Exception as e:
            print(e)

          
ua = t[~t.allocated]
first_bd  = ua.groupby(['first_dm','last_dm']).min().reset_index().set_index(['first_dm','last_dm', 'birthday']).copy()
first_bd['allocated'] = True
first_bd.head()

t = t.reset_index().set_index(['first_dm','last_dm', 'birthday']).join(first_bd['allocated'], rsuffix='_other')
t['allocated'] = t.apply(lambda r: True if r.allocated_other == True else r.allocated, axis = 1)

t.drop(columns='allocated_other', inplace = True)

t.sort_values(['first_dm','last_dm']).to_csv('user_table.csv')

marked = t.copy()
marked.drop_duplicates(subset=['sound_key','bd'], inplace=True)

marked['user_no'] = marked.reset_index().index

allocated =  marked[marked.allocated]

users = allocated.reset_index()[['first','last','birthday','sound_key','user_no']]


/Users/eric/proj/virt/data-project/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2817: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
/Users/eric/proj/virt/data-project/lib/python3.6/site-packages/IPython/core/async_helpers.py:67: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


**********************************************************************
File "/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/tablecloth.io/tablecloth.io-storytent-201811/pylib/metaphone.py", line 19, in __main__
Failed example:
    dm(u'Bartoš'), dm(u'Bartosz'), dm(u'Bartosch'), dm(u'Bartos')
Expected:
    (('PRT', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
Got:
    (('PRTS', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
**********************************************************************
1 items had failures:
   1 of   4 in __main__
***Test Failed*** 1 failures.


In [6]:

s = pkg.resource('source').dataframe()
s.columns = ['#',
 'location',
 "first",
 "last",
 "sex",
 'grade',
 'birthday',
 'Which emoji describes how you feel about reading right now?',
 'start_date',
 'submit_date',
 'Network ID',
 'Form',
 'ClientID',
 'Tablecloth Link',
 'QR Code Link']

s['first'] = s['first'].str.strip().str.title()
s['last'] = s['last'].str.strip().str.title()

s['sound_key'] = s.apply(lambda r: "{}/{}".format(dm(r['last'])[0], dm(r['first'])[0]), axis=1)

t = s.merge(users, on='sound_key', suffixes=('','_r'), how='left')

merged = t[['sound_key', 'user_no', 'last','last_r','first','first_r','birthday','birthday_r',
            'location','sex','grade','start_date','submit_date']].copy()

def mismatch(r):
    f1, f2 = r['first'], r['first_r']
    l1, l2 = r['last'], r['last_r'] 
    
    if f1=='' and l1 =='':
        return 'empty'
    
    if l1!=l2 and f1!=f2:
        return 'both'
    elif l1!=l2:
        return 'last'
    elif f1!=f2:
        return 'first'
    else:
        return ''
    

merged['mismatch'] = merged.apply(mismatch, axis = 1)

merged = merged[merged.mismatch!='empty'].copy().sort_values(['last_r','first_r'])

merged.loc[merged.user_no.isna(),'mismatch'] = 'sound_key'

merged.fillna('', inplace=True)

merged.head()

,sound_key,user_no,last,last_r,first,first_r,birthday,birthday_r,location,sex,grade,start_date,submit_date,mismatch
137,/TRN,561,1,1,Darian,Darian,9/4/2018,2018-09-04,Waypoint Public,Boy,3,9/5/2018 1:52:46,9/5/2018 1:53:54,
146,/JTN,277,1,1,Jayden,Jayden,9/4/2018,2018-09-04,Waypoint Public,Boy,1,9/5/2018 0:24:05,9/5/2018 0:25:27,
318,/JTN,277,1,1,Jayden,Jayden,,2018-09-04,,,,9/5/2018 0:50:55,9/5/2018 0:51:18,
143,/MRK,398,1,1,Margaux,Margaux,9/4/2017,2017-09-04,Waypoint Public,Girl,1,9/5/2018 0:51:40,9/5/2018 0:52:34,
1209,A/ALSP0,68,A,A,Elizabeth,Elizabeth,2/7/2014,2014-02-07,El Cajon,Girl,0,,10/28/2018 20:22:11,


In [7]:
locations = merged[['sound_key','location']].copy()
print(len(locations))
locations = locations[locations['location'].str.strip() != '']
print(len(locations))
locations['n'] = 1
t =locations.drop_duplicates().groupby(['sound_key']).count()
t = t.sort_values('n',ascending=False)
t = t[t.n > 1]
t = merged[merged.sound_key.isin(list(t.index))].sort_values('sound_key')
t.to_csv('multi_location.csv')

1383
776


In [8]:
merged.head()

,sound_key,user_no,last,last_r,first,first_r,birthday,birthday_r,location,sex,grade,start_date,submit_date,mismatch
137,/TRN,561,1,1,Darian,Darian,9/4/2018,2018-09-04,Waypoint Public,Boy,3,9/5/2018 1:52:46,9/5/2018 1:53:54,
146,/JTN,277,1,1,Jayden,Jayden,9/4/2018,2018-09-04,Waypoint Public,Boy,1,9/5/2018 0:24:05,9/5/2018 0:25:27,
318,/JTN,277,1,1,Jayden,Jayden,,2018-09-04,,,,9/5/2018 0:50:55,9/5/2018 0:51:18,
143,/MRK,398,1,1,Margaux,Margaux,9/4/2017,2017-09-04,Waypoint Public,Girl,1,9/5/2018 0:51:40,9/5/2018 0:52:34,
1209,A/ALSP0,68,A,A,Elizabeth,Elizabeth,2/7/2014,2014-02-07,El Cajon,Girl,0,,10/28/2018 20:22:11,


In [9]:
t = merged[['sound_key','last_r','first_r','birthday_r','location','sex','grade']]
t = t[ ((t['last_r'] != '') & (t.first_r != '')) ]
t = t[ (~t['last_r'].isnull()) & (~t['first_r'].isnull()) ]
t.columns = ['sound_key','last_name', 'first_name', 'birthday', 'mode_location', 'sex', 'grade']
t.head()

,sound_key,last_name,first_name,birthday,mode_location,sex,grade
137,/TRN,1,Darian,2018-09-04,Waypoint Public,Boy,3
146,/JTN,1,Jayden,2018-09-04,Waypoint Public,Boy,1
318,/JTN,1,Jayden,2018-09-04,,,
143,/MRK,1,Margaux,2017-09-04,Waypoint Public,Girl,1
1209,A/ALSP0,A,Elizabeth,2014-02-07,El Cajon,Girl,0


In [10]:
def most(x):
    """Select the most common item that isn't empty"""
    try:
        return list(e for e in x.value_counts().index if e and e.strip())[0]
    except AttributeError:
        return list(e for e in x.value_counts().index if e)[0]
    except IndexError:
        return None


t2 = t.groupby(['sound_key']).agg(
    {
        'last_name': most,
        'first_name': most,
        'birthday': 'min',
        'mode_location': most,
        'sex': most, 
        'grade': 'max'
        
    }
)
t2['grade'] = t.groupby(['sound_key']).grade.max()

t2['birthday'] = pd.to_datetime(t2.birthday, errors='coerce')

t2['grade'] = t2.grade.astype(int)

t2['b_error'] = ''

t2['birthday_orig'] = t2.birthday

def birthday_error(r):

    if not r.birthday:
        return 'no_bd'

    if r.grade and r.grade > 0:
        if r.birthday > datetime(2018-5-r.grade, 9, 1):
            return 'too_young_for_grade'
    else:
        if r.birthday.year > 2016:
            return 'too_young'

    return ''
        

t2['bd_error'] = t2.apply(birthday_error, axis=1)

t2['birthday'] = t2.birthday.where(t2.bd_error == '', None)
t2 = t2.reset_index()
t2.insert(0, 'user_id', t2.reset_index().index+1)

user_table=t2
user_table

,user_id,sound_key,last_name,first_name,birthday,mode_location,sex,grade,b_error,birthday_orig,bd_error
0,1,/JTN,1,Jayden,NaT,Waypoint Public,Boy,1,,2018-09-04,too_young_for_grade
1,2,/KLN,H,Collin,NaT,Waypoint Public,Boy,2,,2018-12-04,too_young_for_grade
2,3,/MRK,1,Margaux,NaT,Waypoint Public,Girl,1,,2017-09-04,too_young_for_grade
3,4,/N0N,H,Nathan,NaT,Waypoint Public,Boy,0,,2018-12-04,too_young
4,5,/TLR,H,Taylor,NaT,Waypoint Public,Girl,0,,2018-12-04,too_young
5,6,/TRN,1,Darian,NaT,Waypoint Public,Boy,3,,2018-09-04,too_young_for_grade
6,7,A/AF,A,Eve,2010-05-31,El Cajon,Girl,3,,2010-05-31,
7,8,A/AFN,Wu,evanwu,2011-05-14,City Heights,Boy,2,,2011-05-14,
8,9,A/ALSP0,A,Elizabeth,2014-02-07,El Cajon,Girl,0,,2014-02-07,
9,10,A/KR,A,Gra,NaT,Vista,Girl,0,,2018-09-15,too_young


In [10]:
oci = pkg.resource('orig_checkins').dataframe()
oci['sound_key'] = oci.apply(lambda r: "{}/{}".format(dm(r.last_name)[0],dm(r.first_name)[0]), axis=1)

oci.head()

,type,location,time_in,time_out,duration,feeling_in,feeling_out,first_name,last_name,full_name,name_key,user_id,session_id,checkin_row_no,checkout_row_no,sound_key
0,c,ElCajon,2017-10-25 15:29:23,2017-10-25 16:53:13,84,heart eyes,heart eyes,Aelen,Putrus,,Aelen-Putrus-2011-04-13,5b4b7d42ba8715bf88c384694ecda9c7,a5990f9e-11a9-4e9a-9d8c-ca96cc5a0a87,1,6658,PTRS/ALN
1,i,CityHeights,2018-09-01 08:19:19,,,heart eyes,,Benjamin,Kolee,,Benjamin-Kolee-2018-08-21,7e8c8cdcdcf72e219ede9108da2aef8b,13f0c859-4ab3-44c2-a234-170d16c5f088,2,,KL/PNJMN
2,i,CityHeights,2018-09-01 08:20:33,,,smiling face,,Mary,Kolee,,Mary-Kolee-na,1666804a08d199c707754606e3c988b5,9c800f35-6c93-4587-a179-ef8eb5aaa421,3,,KL/MR
3,i,Vista,2018-09-01 08:21:44,,,smiling face,,Elias,Cheng,,Elias-Cheng-2010-07-12,157fa4828f5e662d869a5ab66d0ac3c7,211f8a88-4bba-49af-a188-b29b38a5124b,4,,XNK/ALS
4,i,CityHeights,2018-09-01 08:22:30,,,smiling face,,Javier,Mendez,,Javier-Mendez-2014-02-16,256399200dd111398947757ce65bbdad,26fae1c1-6df3-47d6-96a7-7d28201d49fa,5,,MNTS/JF
